## Mercedes-Benz Greener Manufacturing.
### Goal: reduce the time a Mercedes-Benz spends on the test bench.
Problem Statement Scenario: Since the first automobile, the Benz Patent Motor Car in 1886, Mercedes-Benz has stood for important automotive innovations. These include the passenger safety cell with a crumple zone, the airbag, and intelligent assistance systems. Mercedes-Benz applies for nearly 2000 patents per year, making the brand the European leader among premium carmakers. Mercedes-Benz is the leader in the premium car industry. With a huge selection of features and options, customers can choose the customized Mercedes-Benz of their dreams. To ensure the safety and reliability of every unique car configuration before they hit the road, the company’s engineers have developed a robust testing system. As one of the world’s biggest manufacturers of premium cars, safety and efficiency ar e paramount on Mercedes-Benz’s production lines. However, optimizing the speed of their testing system for many possible feature combinations is complex and time-consuming without a powerful algorithmic approach. You are required to reduce the time that cars spend on the test bench. Others will work with a dataset representing different permutations of features in a Mercedes-Benz car to predict the time it takes to pass testing. Optimal algorithms will contribute to faster testing, resulting in lower carbon dioxide emissions without reducing Mercedes-Benz’s standards.

Following actions should be performed: If for any column(s), the variance is equal to zero, then you need to remove those variable(s). Check for null and unique values for test and train sets. Apply label encoder. Perform dimensionality reduction. Predict your test_df values using XGBoost,Ridge,Lasso.

Following actions will be performed:
1.If for any column(s), the variance is equal to zero, then you need to remove those variable(s).
2.Check for null and unique values for test and train sets.
3.Apply label encoder.
4.Perform dimensionality reduction.
5.Predict testvalues using XGBoost.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
train = pd.read_csv('D:/job/AI & ML/AI/Project1/train/train.csv', index_col = 'ID')
test = pd.read_csv('D:/job/AI & ML/AI/Project1/test/test.csv', index_col = 'ID')

In [3]:

for col in train.columns:
    if(train[col].dtype != np.float64 and train[col].dtype != np.int64):
        
        # making a list of unique strings in train and test feature
        unique_train = train[col].unique().tolist()
        unique_test = test[col].unique().tolist()
        
        # making a combined list
        for member in unique_test:
            if member not in unique_train:
                unique_train.append(member)
               
        # mapping with numbers
        map_dict = dict(zip(unique_train, range(len(unique_train))))
        train[col] = train[col].replace(to_replace = map_dict)
        test[col] = test[col].replace(to_replace = map_dict)

In [4]:

train.head()

,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
ID,,,,,,,,,,,,,,,,,,,,,
0,130.81,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
6,88.53,0,1,1,1,0,1,1,0,0,...,1,0,0,0,0,0,0,0,0,0
7,76.26,1,2,2,2,0,2,0,1,0,...,0,0,0,0,0,0,1,0,0,0
9,80.62,1,1,2,3,0,2,1,2,0,...,0,0,0,0,0,0,0,0,0,0
13,78.02,1,0,2,3,0,3,2,3,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
print(train.isnull().values.any())
print(test.isnull().values.any())
train.describe()

False
False


,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
count,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,...,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000
mean,100.669318,12.110715,6.467569,7.851509,2.415301,0.002138,16.839629,3.031124,11.457591,0.013305,...,0.318841,0.057258,0.314802,0.020670,0.009503,0.008078,0.007603,0.001663,0.000475,0.001426
std,12.679381,8.315637,4.789927,5.644031,1.361654,0.073900,6.357474,2.554581,7.040194,0.114590,...,0.466082,0.232363,0.464492,0.142294,0.097033,0.089524,0.086872,0.040752,0.021796,0.037734
min,72.110000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,90.820000,6.000000,3.000000,4.000000,2.000000,0.000000,11.000000,1.000000,5.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,99.150000,11.000000,6.000000,7.000000,2.000000,0.000000,17.000000,2.000000,12.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,109.010000,15.000000,7.000000,10.000000,3.000000,0.000000,22.000000,6.000000,17.000000,0.000000,...,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,265.320000,46.000000,26.000000,43.000000,6.000000,3.000000,28.000000,11.000000,24.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
X_train = train.iloc[:,1:]
y_train = train.iloc[:,0]

from sklearn.preprocessing import MinMaxScaler
scalling = MinMaxScaler().fit(X_train)
X_train_scalled = scalling.transform(X_train)
test_scalled = scalling.transform(test)

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

reg = LinearRegression()
print(cross_val_score(reg, X_train_scalled, y_train, cv=10))

[-1.36765404e+21 -3.80496550e+18 -1.23861964e+20 -3.76773521e+19
 -6.45164466e+15 -4.18090587e+19 -2.75279312e+20 -1.41088404e+20
 -2.51704139e+19 -4.04043030e+20]


In [8]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

grid_values = {'alpha': [0.0235, 0.024, 0.0245]}
grid_lasso = GridSearchCV(Lasso(), param_grid = grid_values, cv=10, scoring = 'r2')
grid_lasso.fit(X_train_scalled, y_train)
predict_lasso = grid_lasso.predict(test_scalled)

print('Mean score matrix: ', grid_lasso.cv_results_['mean_test_score'])
print('Grid best parameter (max. accuracy): ', grid_lasso.best_params_)
print('Grid best score (accuracy): ', grid_lasso.best_score_)

Mean score matrix:  [0.56300014 0.56300502 0.56299962]
Grid best parameter (max. accuracy):  {'alpha': 0.024}
Grid best score (accuracy):  0.5630050235597193


In [9]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

grid_values = {'alpha': [40, 40.5, 41]}
grid_ridge = GridSearchCV(Ridge(), param_grid = grid_values, cv=10, scoring = 'r2')
grid_ridge.fit(X_train_scalled, y_train)
predict_ridge = grid_ridge.predict(test_scalled)

print('Mean score matrix: ', grid_ridge.cv_results_['mean_test_score'])
print('Grid best parameter (max. accuracy): ', grid_ridge.best_params_)
print('Grid best score (accuracy): ', grid_ridge.best_score_)

Mean score matrix:  [0.55378705 0.55378761 0.55378743]
Grid best parameter (max. accuracy):  {'alpha': 40.5}
Grid best score (accuracy):  0.5537876100313108


In [10]:
import xgboost as xgb

grid_values = {'n_estimators': [74,75,76], 'learning_rate': [0.13,0.135,0.14], 'max_depth': [1,2,3]}
grid_xgb = GridSearchCV(xgb.XGBRegressor(), param_grid = grid_values, cv=10, scoring = 'r2')
grid_xgb.fit(X_train_scalled, y_train)
predict_xgb = grid_xgb.predict(test_scalled)

print('Mean score matrix: ', grid_xgb.cv_results_['mean_test_score'])
print('Grid best parameter (max. accuracy): ', grid_xgb.best_params_)
print('Grid best score (accuracy): ', grid_xgb.best_score_)

Mean score matrix:  [0.5559935  0.55622105 0.55678005 0.58167739 0.58183296 0.58178903
 0.57940737 0.57943899 0.57941318 0.55723419 0.55748037 0.55776755
 0.58207435 0.58216552 0.58204784 0.5772963  0.57723216 0.57716378
 0.55809566 0.55818499 0.55838828 0.58156077 0.58151575 0.58169073
 0.57786969 0.57786955 0.57786837]
Grid best parameter (max. accuracy):  {'learning_rate': 0.135, 'max_depth': 2, 'n_estimators': 75}
Grid best score (accuracy):  0.58216551862013


In [11]:
final_predictions = pd.DataFrame()
final_predictions['id'] = test.index
final_predictions['y'] = pd.Series(predict_xgb)
final_predictions.to_csv('predictions.csv', index=False)

In [12]:
final_predictions

,id,y
0,1,77.395775
1,2,92.331261
2,3,77.376266
3,4,77.042000
4,5,112.433716
...,...,...
4204,8410,101.806328
4205,8411,91.542648
4206,8413,91.932625
4207,8414,110.443840
